In [16]:
import sympy as sp
from utils import rz, tx, tz

## Variable definition

In [17]:
a1, a2, q1, q2, q3 = sp.symbols("a1 a2 q1 q2 q3")
q = [q1, q2, q3]

## Transformation definition
The SCARA robot is a RRT, defined as:
$$
M = R_z(q_1) \cdot T_x(a_1) \cdot  R_z(q_2) \cdot T_x(a_2) \cdot  T_z(q_3)
$$

In [18]:
m = sp.simplify(rz(q1) * tx(a1) * rz(q2) * tx(a2) * tz(q3))
m

Matrix([
[ cos(q1 + q2), sin(q1 + q2), 0,  a1*cos(q1) + a2*cos(q1 + q2)],
[-sin(q1 + q2), cos(q1 + q2), 0, -a1*sin(q1) - a2*sin(q1 + q2)],
[            0,            0, 1,                            q3],
[            0,            0, 0,                             1]])

## Position

In [19]:
edge = sp.Matrix([0, 0, 0, 1]) 
p = sp.simplify((m @ edge))[:3, :]
p

Matrix([
[ a1*cos(q1) + a2*cos(q1 + q2)],
[-a1*sin(q1) - a2*sin(q1 + q2)],
[                           q3]])

## Jacobian

In [20]:
jacobian = sp.simplify(p.jacobian(q))
jacobian

Matrix([
[-a1*sin(q1) - a2*sin(q1 + q2), -a2*sin(q1 + q2), 0],
[-a1*cos(q1) - a2*cos(q1 + q2), -a2*cos(q1 + q2), 0],
[                            0,                0, 1]])

## Inverse Jacobian

In [21]:
jacobian_inv = sp.simplify(jacobian.inv())
jacobian_inv

Matrix([
[                      cos(q1 + q2)/(a1*sin(q2)),                     -sin(q1 + q2)/(a1*sin(q2)), 0],
[-(a1*cos(q1) + a2*cos(q1 + q2))/(a1*a2*sin(q2)), (a1*sin(q1) + a2*sin(q1 + q2))/(a1*a2*sin(q2)), 0],
[                                              0,                                              0, 1]])

# Pseudo Inverse

In [24]:
sp.simplify(jacobian.pinv())

Matrix([
[                      cos(q1 + q2)/(a1*sin(q2)),                     -sin(q1 + q2)/(a1*sin(q2)), 0],
[-(a1*cos(q1) + a2*cos(q1 + q2))/(a1*a2*sin(q2)), (a1*sin(q1) + a2*sin(q1 + q2))/(a1*a2*sin(q2)), 0],
[                                              0,                                              0, 1]])

## Pseudo Inverse Regularized

In [ ]:
n, m = jacobian.shape
identity = sp.eye(m)

lam = sp.symbols('lambda', positive=True)
jacobian_pinv_reg = sp.simplify((jacobian.T * jacobian + lam * identity).inv() * jacobian.T)
jacobian_pinv_reg

Matrix([
[                                                (a1*a2**2*(sin(q1 + 2*q2) + sin(3*q1 + 2*q2))/4 - a1*a2**2*sin(q1)*cos(q1 + q2)**2 - a1*lambda*sin(q1) - a2*lambda*sin(q1 + q2))/(a1**2*a2**2*sin(q2)**2 + a1**2*lambda + 2*a1*a2*lambda*cos(q2) + 2*a2**2*lambda + lambda**2),                                                 (a1*a2**2*(cos(q1 + 2*q2) - cos(3*q1 + 2*q2))/4 - a1*a2**2*sin(q1 + q2)**2*cos(q1) - a1*lambda*cos(q1) - a2*lambda*cos(q1 + q2))/(a1**2*a2**2*sin(q2)**2 + a1**2*lambda + 2*a1*a2*lambda*cos(q2) + 2*a2**2*lambda + lambda**2),              0],
[a2*(a1**2*(sin(q1 - q2) + sin(3*q1 + q2))/4 - a1**2*sin(q1 + q2)*cos(q1)**2 - a1*a2*(sin(q1 + 2*q2) + sin(3*q1 + 2*q2))/4 + a1*a2*sin(q1)*cos(q1 + q2)**2 - lambda*sin(q1 + q2))/(a1**2*a2**2*sin(q2)**2 + a1**2*lambda + 2*a1*a2*lambda*cos(q2) + 2*a2**2*lambda + lambda**2), a2*(a1**2*(cos(q1 - q2) - cos(3*q1 + q2))/4 - a1**2*sin(q1)**2*cos(q1 + q2) - a1*a2*(cos(q1 + 2*q2) - cos(3*q1 + 2*q2))/4 + a1*a2*sin(q1 + q2)**2*cos(q1) - 

In [26]:
n, m = jacobian.shape
identity = sp.eye(m)

lam = 1e-1
jacobian_pinv_reg = sp.simplify((jacobian.T * jacobian + lam * identity).inv() * jacobian.T)
jacobian_pinv_reg

Matrix([
[                                                          1.0*(0.25*a1*a2**2*(sin(q1 + 2*q2) + sin(3*q1 + 2*q2)) - 1.0*a1*a2**2*sin(q1)*cos(q1 + q2)**2 - 0.1*a1*sin(q1) - 0.1*a2*sin(q1 + q2))/(1.0*a1**2*a2**2*sin(q2)**2 + 0.1*a1**2 + 0.2*a1*a2*cos(q2) + 0.2*a2**2 + 0.01),                                                           1.0*(0.25*a1*a2**2*(cos(q1 + 2*q2) - cos(3*q1 + 2*q2)) - 1.0*a1*a2**2*sin(q1 + q2)**2*cos(q1) - 0.1*a1*cos(q1) - 0.1*a2*cos(q1 + q2))/(1.0*a1**2*a2**2*sin(q2)**2 + 0.1*a1**2 + 0.2*a1*a2*cos(q2) + 0.2*a2**2 + 0.01),                 0],
[1.0*a2*(0.25*a1**2*(sin(q1 - q2) + sin(3*q1 + q2)) - 1.0*a1**2*sin(q1 + q2)*cos(q1)**2 - 0.25*a1*a2*(sin(q1 + 2*q2) + sin(3*q1 + 2*q2)) + 1.0*a1*a2*sin(q1)*cos(q1 + q2)**2 - 0.1*sin(q1 + q2))/(1.0*a1**2*a2**2*sin(q2)**2 + 0.1*a1**2 + 0.2*a1*a2*cos(q2) + 0.2*a2**2 + 0.01), 1.0*a2*(0.25*a1**2*(cos(q1 - q2) - cos(3*q1 + q2)) - 1.0*a1**2*sin(q1)**2*cos(q1 + q2) - 0.25*a1*a2*(cos(q1 + 2*q2) - cos(3*q1 + 2*q2)) + 1.0*a1*a2*si